In [13]:
import pandas as pd
import json

# Load the JSON file
with open('reddit_submissions.json', 'r') as f:
    data = json.load(f)

submissions = data['submissions']

# Create DataFrame from the submissions data
df = pd.DataFrame(submissions)

# drop rows where selftext is an empty string
df = df[df['selftext'] != ''].reset_index()

df = df[["author", "created_utc", "title", "selftext", "score", "upvote_ratio", "num_comments", "num_crossposts", "url"]]

print(len(df))

df

14056


,author,created_utc,title,selftext,score,upvote_ratio,num_comments,num_crossposts,url
0,minerva_K,1340259437,is this thing on?,anybody awake?,3,NaN,2,NaN,http://www.reddit.com/r/Menopause/comments/vdc...
1,[deleted],1351567414,Turned 40 last July and started have hot flash...,My mother said she went into full blown menopa...,1,NaN,0,NaN,http://www.reddit.com/r/Menopause/comments/12b...
2,[deleted],1353294608,winter shoes for mom,my mom has osteoarthritis and i want to get he...,1,NaN,0,NaN,http://www.reddit.com/r/Menopause/comments/13f...
3,oldmanjoe,1356649991,Sex after Menopause?,"My wife went through this ""wonderful"" change, ...",7,NaN,3,NaN,http://www.reddit.com/r/Menopause/comments/15j...
4,[deleted],1362754970,How does one live with someone going through M...,My wife (early 50s and has been diagnosed by h...,3,NaN,6,NaN,http://www.reddit.com/r/Menopause/comments/19w...
...,...,...,...,...,...,...,...,...,...
14051,Rosie4268,1672524104,I'm so over this moodiness,I'm in peri and have been for some time. Moodi...,33,0.96,20,0.0,https://www.reddit.com/r/Menopause/comments/10...
14052,pumaboxbug2,1672525367,How can we make 2023 kinder to us than 2022?,What are non-medicated ways to ease the depres...,14,1.00,17,0.0,https://www.reddit.com/r/Menopause/comments/10...
14053,Street-Lab-9570,1672526636,Hot flashes,I’m 54 post meno and taking estrodot 75 mg and...,3,1.00,3,0.0,https://www.reddit.com/r/Menopause/comments/10...
14054,One-Reflection-6779,1672528757,Perimenopause - change in vaginal odor?,"Hi everyone,\n\nHas anyone else noticed a chan...",23,0.97,24,0.0,https://www.reddit.com/r/Menopause/comments/10...
